## Penggunaan Library

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import  StringLookup, CategoryEncoding
import numpy as np
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mlb
import plotly.express as px


## Data Understanding dan Data Preprocessing

In [ ]:
df = pd.read_csv("DataDestinasi.csv")
df.head(5)

,Place_Id,Place_Name,Description,Unnamed: 3,City,Price,Rating,Alamat Detail,Category
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,NaN,Jakarta,20000,4.6,"Jl. Medan Merdeka Barat, Gambir, Jakarta Pusat",Sejarah dan Budaya
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",NaN,Jakarta,0,4.6,"Jl. Taman Fatahillah, Pinangsia, Jakarta Barat",Sejarah dan Budaya
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,NaN,Jakarta,270000,4.6,"Jl. Lodan Timur No.7, Ancol, Jakarta Utara",Taman Hiburan
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,NaN,Jakarta,10000,4.5,"Jl. Taman Mini Indonesia Indah, Cipayung, Jaka...",Sejarah dan Budaya
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,NaN,Jakarta,94000,4.5,"Jl. Lodan Timur No.7, Ancol, Jakarta Utara",Taman Air


In [ ]:
df.shape

(574, 9)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Place_Id       574 non-null    int64  
 1   Place_Name     574 non-null    object 
 2   Description    573 non-null    object 
 3   Unnamed: 3     117 non-null    object 
 4   City           574 non-null    object 
 5   Price          574 non-null    int64  
 6   Rating         574 non-null    float64
 7   Alamat Detail  574 non-null    object 
 8   Category       574 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 40.5+ KB


In [ ]:
df.describe()

,Place_Id,Price,Rating
count,574.000000,574.000000,574.000000
mean,287.500000,24139.372822,4.429791
std,165.843802,60432.391228,0.207707
min,1.000000,0.000000,3.400000
25%,144.250000,0.000000,4.300000
50%,287.500000,7500.000000,4.400000
75%,430.750000,20000.000000,4.600000
max,574.000000,900000.000000,5.000000


In [ ]:
from statistics import mode
rating = df["Rating"]
kategori = df["Category"]
harga = df["Price"]
print(mode(harga))
print(mode(rating))
print(mode(kategori))

0
4.5
Wisata Alam


In [ ]:
#missing value check
df.isnull().sum()

,0
Place_Id,0
Place_Name,0
Description,1
Unnamed: 3,457
City,0
Price,0
Rating,0
Alamat Detail,0
Category,0


In [ ]:
# check duplicate
df1 = df.apply(lambda x:sum(x.duplicated()))
print(df1)

Place_Id           0
Place_Name        12
Description        0
Unnamed: 3       536
City             563
Price            519
Rating           560
Alamat Detail     83
Category         546
dtype: int64


In [ ]:
# Check missing value
df.isna().sum()

,0
Place_Id,0
Place_Name,0
Description,1
Unnamed: 3,457
City,0
Price,0
Rating,0
Alamat Detail,0
Category,0


## Pembuatan function untuk label encoder dan decoder

In [ ]:
def create_label_mappings(csv_df):
    # Baca file CSV ke dalam dataframe
    df = csv_df

    # Buat label_encoder: Dictionary berbasis nama tempat
    label_encoder = {
        row['Place_Name']: {
            "index": idx,
            "Price": row["Price"],
            "Category": row["Category"],
            "Rating": row["Rating"],
            "City": row["City"]
        }
        for idx, row in df.iterrows()
    }

    # Buat label_decoder: Dictionary berbasis indeks
    label_decoder = {
        value["index"]: {
            "Place_Name": key,
            "Price": value["Price"],
            "Category": value["Category"],
            "Rating": value["Rating"],
            "City": value["City"]
        }
        for key, value in label_encoder.items()
    }

    return label_encoder, label_decoder

label_encoder, label_decoder = create_label_mappings(df)

## Pembuatan arsitekture neural network

In [ ]:
# Normalizer untuk harga
price_normalizer = layers.Normalization()
price_normalizer.adapt(df['Price'].values.reshape(-1, 1))

# Normalizer untuk rating
rating_normalizer = layers.Normalization()
rating_normalizer.adapt(df['Rating'].values.reshape(-1, 1))
# Input untuk rating
rating_input = layers.Input(shape=(1,), dtype=tf.float32, name="rating")
rating_scaled = rating_normalizer(rating_input)
# String lookup untuk kota
unique_cities = df['City'].unique()
city_lookup = layers.StringLookup(vocabulary=unique_cities)

# Category encoding untuk kota
city_encoder = layers.CategoryEncoding(num_tokens=len(unique_cities) + 1, output_mode="one_hot")

# String lookup untuk kategori
unique_categories = df['Category'].unique()
category_lookup = StringLookup(vocabulary=unique_categories)

# Category encoding untuk kategori
category_encoder = CategoryEncoding(num_tokens=len(unique_categories) + 1, output_mode="one_hot")


# # Label encoding untuk tempat
label_encoder = {place: idx for idx, place in enumerate(df['Place_Name'].unique())}
df['Place_Label'] = df['Place_Name'].map(label_encoder)

# Input untuk model
price_input = layers.Input(shape=(1,), dtype=tf.float32, name="price")
city_input = layers.Input(shape=(1,), dtype=tf.string, name="city")
category_input = layers.Input(shape=(1,), dtype=tf.string, name="category")

# Preprocessing di dalam model
price_scaled = price_normalizer(price_input)
city_index = city_lookup(city_input)
city_one_hot = city_encoder(city_index)
category_index = category_lookup(category_input)
category_one_hot = category_encoder(category_index)

# Menggabungkan preprocessing
preprocessed_input = layers.Concatenate()([price_scaled, city_one_hot, rating_scaled, category_one_hot])

# Arsitektur model
x = layers.Dense(256, activation='relu')(preprocessed_input)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(len(label_encoder), activation='softmax')(x)


model = Model(inputs=[price_input, city_input, rating_input, category_input], outputs=output)

# Kompilasi model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data latih (mengonversi 'Price' menjadi float)
train_prices = df['Price'].values.astype(float)
train_cities = df['City'].values
train_ratings = df['Rating'].values.astype(float)
train_categories = df['Category'].values
train_labels = df['Place_Label'].values

# Latih model
model.fit([train_prices, train_cities, train_ratings, train_categories], train_labels, epochs=300, batch_size=32)

# Simpan model
model.save("model_destinasi_wisata_with_preprocessing.h5")


Epoch 1/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0000e+00 - loss: 6.3383
Epoch 2/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0114 - loss: 6.3118     
Epoch 3/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0231 - loss: 6.2630     
Epoch 4/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0341 - loss: 6.1320
Epoch 5/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0292 - loss: 5.7326
Epoch 6/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0351 - loss: 5.1351 
Epoch 7/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0651 - loss: 4.4130
Epoch 8/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1510 - loss: 3.6880 
Epoch 9/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2033 - loss: 3.1202
Epoch 10/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2922 - loss: 2.7581 
Epoch 11/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3620 - loss: 2.3766 
Epoch 12/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

## Pembuatan function

In [ ]:
def predict_destination(price, rating, city, category, top_n=5):
    # Pastikan input berada dalam bentuk tensor dengan tipe data yang sesuai
    price_input = tf.constant([price], dtype=tf.float32)
    rating_input = tf.constant([rating], dtype=tf.float32)
    city_input = tf.constant([city], dtype=tf.string)
    category_input = tf.constant([category], dtype=tf.string)

    # Prediksi probabilitas
    probabilities = model.predict([price_input, city_input, rating_input, category_input])[0]

    # Ambil indeks dengan probabilitas tertinggi
    top_indices = np.argsort(probabilities)[::-1]  # Urutkan skor tertinggi ke terendah

    # Filter awal dengan kategori
    recommendations = []
    for idx in top_indices:
        if idx not in label_decoder:
            continue

        # Ambil detail tempat dari label_decoder
        place_data = label_decoder[idx]
        place_name = place_data["Place_Name"]
        place_city = place_data["City"]
        place_price = place_data["Price"]
        place_category = place_data["Category"]

        # Terapkan filter awal
        if place_city.lower() == city.lower() and place_price < price and place_category.lower() == category.lower():
            recommendations.append({
                "Place_Name": place_name,
                "City": place_city,
                "Price": int(place_price),
                "Category": place_category,
            })

        if len(recommendations) == top_n:
            break

    # Jika rekomendasi masih kosong, abaikan filter kategori
    if len(recommendations) == 0:
        for idx in top_indices:
            if idx not in label_decoder:
                continue

            # Ambil detail tempat dari label_decoder
            place_data = label_decoder[idx]
            place_name = place_data["Place_Name"]
            place_city = place_data["City"]
            place_price = place_data["Price"]
            place_category = place_data["Category"]

            # Terapkan filter tanpa kategori
            if place_city == city and place_price < price:
                recommendations.append({
                    "Place_Name": place_name,
                    "City": place_city,
                    "Price": int(place_price),
                    "Category": place_category,
                })

            if len(recommendations) == top_n:
                break

    return recommendations


predicted_places = predict_destination(price=40000, rating=4, city="Jakarta", category="museum")

for place in predicted_places:
    print(place)


NameError: name 'tf' is not defined

## Save model

In [ ]:
from tensorflow.keras.models import load_model

# Memuat model dari file H5
loaded_model = load_model("model_destinasi_wisata_with_preprocessing.h5")


## Install Tensorflow JS

In [ ]:
!pip install tensorflowjs

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 84.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      

## Convert h5 model ke tensorflow JS

In [ ]:
!tensorflowjs_converter --input_format=keras model_destinasi_wisata.h5 ./model_directory

/bin/bash: line 1: tensorflowjs_converter: command not found


In [ ]:
def recommend_places(price, city, rating, category, top_n=5):
    # Filter data sesuai dengan kota
    city_filtered = df[df['City'] == city]

    # Jika tidak ada data sesuai kota, kembalikan pesan
    if city_filtered.empty:
        return f"No places found for city: {city}"

    # Ambil indeks dari data yang difilter
    filtered_indices = city_filtered.index.tolist()
    filtered_places_info = city_filtered[['Place_Name', 'City', 'Price', 'Category']].to_numpy()

    # Konversi input
    price_input = tf.constant([price], dtype=tf.float32)
    city_input = tf.constant([city], dtype=tf.string)
    rating_input = tf.constant([rating], dtype=tf.float32)
    category_input = tf.constant([category], dtype=tf.string)

    # Prediksi hanya untuk data yang difilter
    probabilities = model.predict([price_input, city_input, rating_input, category_input])[0]

    # Ambil prediksi untuk data yang difilter
    filtered_probabilities = probabilities[filtered_indices]
    top_indices = np.argsort(filtered_probabilities)
    print(top_indices)
    # Ambil informasi tempat langsung dari filtered_places_info
    recommendations = []
    for idx in top_indices:
        place_name, place_city, place_price, place_category = filtered_places_info[idx]
        if place_city == city and place_price < price and place_category == category :
          recommendations.append({
              "Place_Name": place_name,
              "City": place_city,
              "Price": int(place_price),
              "Category": place_category
          })
        if len(recommendations) == top_n:
          break

    return recommendations

# Prediksi dengan fitur tambahan kategori
result = recommend_places(30000, "Bandung",4, "Wisata Alam", top_n=3)
for i in result:
    print(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
[141 127 124  42  91  47  51  53 121  57  58  59  63 114  37  67  69 112
 108  72 107  74  77  78  79  81  82  84  87  68 128  92  96  22   4   2
 131  25  14  27 139  12  10   8   1  29 102  45 105  35   7  16 136  11
  94 133  62 132 138  28  23  95 103  44 135 104 137 125  89  43  17 117
 120  36 116  24  93 123  40 122  52  30  99  88 118 130  75  85 134   9
  21  48 101  98  50  49  15  34  54   0 106 113 126  90  55  86  19  20
  61   5  97 110  66  60 119 115   3  71  56  33  65 140  41  46 129  83
  73  31  76  39  64  32  26  13  38  18  70 100 111   6 109  80]
{'Place_Name': 'Kawah Putih', 'City': 'Bandung', 'Price': 20000, 'Category': 'Wisata Alam'}
{'Place_Name': 'Selasar Sunaryo Art Space', 'City': 'Bandung', 'Price': 25000, 'Category': 'Wisata Alam'}
{'Place_Name': 'Masjid Pusdai', 'City': 'Bandung', 'Price': 15000, 'Category': 'Wisata Alam'}
